In [40]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, f1_score
from pandas import Series, DataFrame
import scipy
from scipy import stats
import math

In [2]:
%matplotlib inline

In [3]:
train = pd.read_csv("Dataset-football-train.txt", delimiter="\t")

In [4]:
train.head(24)

,ID,Date,Opponent,Is_Home_or_Away,Is_Opponent_in_AP25_Preseason,Media,Label
0,1,9/5/15,Texas,Home,Out,NBC,Win
1,2,9/12/15,Virginia,Away,Out,ABC,Win
2,3,9/19/15,Georgia Tech,Home,In,NBC,Win
3,4,9/26/15,UMass,Home,Out,NBC,Win
4,5,10/3/15,Clemson,Away,In,ABC,Lose
5,6,10/10/15,Navy,Home,Out,NBC,Win
6,7,10/17/15,USC,Home,In,NBC,Win
7,8,10/31/15,Temple,Away,Out,ABC,Win
8,9,11/7/15,PITT,Away,Out,ABC,Win
9,10,11/14/15,Wake Forest,Home,Out,NBC,Win


In [5]:
test = pd.read_csv("Dataset-football-test.txt", delimiter="\t")

In [6]:
test.head()

,ID,Date,Opponent,Is_Home_or_Away,Is_Opponent_in_AP25_Preseason,Media,Label
0,25,9/2/17,Temple,Home,Out,NBC,Win
1,26,9/9/17,Georgia,Home,In,NBC,Lose
2,27,9/16/17,Boston College,Away,Out,ESPN,Win
3,28,9/23/17,Michigan State,Away,Out,FOX,Win
4,29,9/30/17,Miami Ohio,Home,Out,NBC,Win


Here we create the function to check the information gain.

In [7]:
from collections import defaultdict
def information_gain(column_x, column_y, yes_label, no_label):
    yes = 0
    no = 0
    for item in column_y:
        if item == yes_label:
            yes += 1
        else:
            no += 1
    try:
        hy1 = (-float(yes)/len(column_y))* math.log(float(yes)/len(column_y), 2)
    except:
        hy1=0
    try:
        hy2 = (-float(no)/len(column_y))*math.log(float(no)/len(column_y), 2)
    except:
        hy2=0
    H_Y = hy1 + hy2
    options = defaultdict(list)
    for index, item in enumerate(column_x):
        options[item].append(index)
     
    y_given_x = 0
    for item in options:
        y = 0
        n = 0
        for index in options[item]:
            if column_y[index] == yes_label:
                y += 1
            else: 
                n += 1
                
        try:
            y_given_x_yes = (float(y)/len(column_x))*(-float(y)/len(item))* (math.log(float(y)/len(item), 2))
        except:
            y_given_x_yes = 0
        try:
            y_given_x_no = (float(n)/len(column_x))*(-float(n)/len(item))* (math.log(float(n)/len(item), 2))
        except:
            y_given_x_no = 0
        y_given_x += y_given_x_yes + y_given_x_no
    information_gain = abs(H_Y - y_given_x)
    print(information_gain)
    return

In [8]:
Label = list(train["Label"])

The dates are exact, so I decided to sort them into buckets based on month and year. This way if we actually split on date it is information that might be helpful in predicting. Otherwise, since there's no way the team would play two games on one day, the date would never be useful.

In [10]:
i = 0
for item in train["Date"]:
    if item[0] == "9" and item[len(item)-1] =="5":
        train["Date"][i] = "sept_15"
    elif item[0] == "9" and item[len(item)-1] =="6":
        train["Date"][i] = "sept_16"
    elif item[1] == "0" and item[len(item)-1] =="5":
        train["Date"][i] = "oct_15"
    elif item[1] == "0" and item[len(item)-1] =="6":
        train["Date"][i] = "oct_16"
    elif item[1] == "1" and item[len(item)-1] =="5":
        train["Date"][i] = "nov_15"
    else:
        train["Date"][i] = "nov_16"
    i += 1
    

/Users/kailee/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kailee/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kailee/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kailee/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set 

In [11]:
train["Date"]

0     sept_15
1     sept_15
2     sept_15
3     sept_15
4      oct_15
5      oct_15
6      oct_15
7      oct_15
8      nov_15
9      nov_15
10     nov_15
11     nov_15
12    sept_16
13    sept_16
14    sept_16
15    sept_16
16     oct_16
17     oct_16
18     oct_16
19     oct_16
20     nov_16
21     nov_16
22     nov_16
23     nov_16
Name: Date, dtype: object

I turn all of the feature columns into lists so that they are in the proper format to go into my information gain function.

In [12]:
Media = list(train["Media"])
H_A = list(train["Is_Home_or_Away"])
Date = list(train["Date"])
Preseason = list(train["Is_Opponent_in_AP25_Preseason"])
Opponent = list(train["Opponent"])

In [13]:
print("IG for Media is:")
information_gain(Media, Label, "Win", "Lose")
print("IG for H_A is:")
information_gain(H_A, Label, "Win", "Lose")
print("IG for Date is:")
information_gain(Date, Label, "Win", "Lose")
print("IG for Preseason is:")
information_gain(Preseason, Label, "Win", "Lose")
print("IG for Opponent is:")
information_gain(Opponent, Label, "Win", "Lose")

IG for Media is:
3.5119220121
IG for H_A is:
2.52454807158
IG for Date is:
0.491375199749
IG for Preseason is:
5.92426380405
IG for Opponent is:
0.567158476583


Preseason has the highest information gain, so that is what we will branch on. This means that we need to sort all our other values into either branch 1 or branch 2 depending on whether they were in or out of the preseason.

In [15]:
Media1 = []
H_A1 = []
Date1 = []
Opponent1 = []
Label1 = []
Media2 = []
H_A2 = []
Date2 = []
Opponent2 = []
Label2 = []
for index, item in enumerate(Preseason):
    if item == "In":
        Media1.append(Media[index])
        H_A1.append(H_A[index])
        Date1.append(Date[index])
        Opponent1.append(Opponent[index])
        Label1.append(Label[index])
    else:
        Media2.append(Media[index])
        H_A2.append(H_A[index])
        Date2.append(Date[index])
        Opponent2.append(Opponent[index])
        Label2.append(Label[index])
In = pd.DataFrame()
Out = pd.DataFrame()
In["Media"] = Media1
In["H_A"] = H_A1
In["Date"] = Date1
In["Opponent"] = Opponent1
In["Label"] = Label1
Out["Media"] = Media2
Out["H_A"] = H_A2
Out["Date"] = Date2
Out["Opponent"] = Opponent2
Out["Label"] = Label2

In [16]:
In.head()

,Media,H_A,Date,Opponent,Label
0,NBC,Home,sept_15,Georgia Tech,Win
1,ABC,Away,oct_15,Clemson,Lose
2,NBC,Home,oct_15,USC,Win
3,FOX,Away,nov_15,Stanford,Lose
4,NBC,Home,oct_16,Stanford,Lose


In [17]:
print("IG for Media in In is:")
information_gain(Media1, Label1, "Win", "Lose")
print("IG for H_A in In is:")
information_gain(H_A1, Label1, "Win", "Lose")
print("IG for Date in In is:")
information_gain(Date1, Label1, "Win", "Lose")
print("IG for Opponent in In is:")
information_gain(Opponent1, Label1, "Win", "Lose")

IG for Media in In is:
0.453381877644
IG for H_A in In is:
0.44400137237
IG for Date in In is:
0.408691896551
IG for Opponent in In is:
0.428679360789


We will split on Media since this is the highest information gain.

In [18]:
H_A_nbc = []
H_A_abc = []
H_A_fox = []
Label_nbc = []
Label_abc = []
Label_fox = []
Date_nbc = []
Date_abc = []
Date_fox = []
Opponent_nbc = []
Opponent_abc = []
Opponent_fox = []
for index, item in enumerate(Media1):
    if item == "NBC":
        H_A_nbc.append(H_A1[index])
        Date_nbc.append(Date1[index])
        Opponent_nbc.append(Opponent1[index])
        Label_nbc.append(Label1[index])
    elif item == "ABC":
        H_A_abc.append(H_A1[index])
        Date_abc.append(Date1[index])
        Opponent_abc.append(Opponent1[index])
        Label_abc.append(Label1[index])
    else:
        H_A_fox.append(H_A1[index])
        Date_fox.append(Date1[index])
        Opponent_fox.append(Opponent1[index])
        Label_fox.append(Label1[index])

Now we return to the Out branch.

In [19]:
Out.head()

,Media,H_A,Date,Opponent,Label
0,NBC,Home,sept_15,Texas,Win
1,ABC,Away,sept_15,Virginia,Win
2,NBC,Home,sept_15,UMass,Win
3,NBC,Home,oct_15,Navy,Win
4,ABC,Away,oct_15,Temple,Win


In [20]:
print("IG for Media in Out is:")
information_gain(Media2, Label2, "Win", "Lose")
print("IG for H_A in Out is:")
information_gain(H_A2, Label2, "Win", "Lose")
print("IG for Date in Out is:")
information_gain(Date2, Label2, "Win", "Lose")
print("IG for Opponent in Out is:")
information_gain(Opponent2, Label2, "Win", "Lose")

IG for Media in Out is:
2.49746609611
IG for H_A in Out is:
1.70140255558
IG for Date in Out is:
0.376915748979
IG for Opponent in Out is:
0.470218797116


We will split on Media since this is the highest information gain.

In [21]:
H_A_nbc2 = []
H_A_abc2 = []
H_A_cbs2 = []
H_A_espn2 = []
Label_nbc2 = []
Label_abc2 = []
Label_cbs2 = []
Label_espn2 = []
Date_nbc2 = []
Date_abc2 = []
Date_cbs2 = []
Date_espn2 = []
Opponent_nbc2 = []
Opponent_abc2 = []
Opponent_cbs2 = []
Opponent_espn2 = []
for index, item in enumerate(Media2):
    if item == "NBC":
        H_A_nbc2.append(H_A2[index])
        Date_nbc2.append(Date2[index])
        Opponent_nbc2.append(Opponent2[index])
        Label_nbc2.append(Label2[index])
    elif item == "ABC":
        H_A_abc2.append(H_A2[index])
        Date_abc2.append(Date2[index])
        Opponent_abc2.append(Opponent2[index])
        Label_abc2.append(Label2[index])
    elif item == "CBS":
        H_A_cbs2.append(H_A2[index])
        Date_cbs2.append(Date2[index])
        Opponent_cbs2.append(Opponent2[index])
        Label_cbs2.append(Label2[index])
    else:
        H_A_espn2.append(H_A2[index])
        Date_espn2.append(Date2[index])
        Opponent_espn2.append(Opponent2[index])
        Label_espn2.append(Label2[index])

Now we calculate the information gain for these four branches: Out_NBC and Out_ABC. We don't have to worry about Out_CBS and Out_ESPN since they each only have one game contained within them, so there is no possibility of splitting, rather they are the final nodes. 

In [22]:
print(Date2)

['sept_15', 'sept_15', 'sept_15', 'oct_15', 'oct_15', 'nov_15', 'nov_15', 'nov_15', 'sept_16', 'sept_16', 'sept_16', 'sept_16', 'oct_16', 'oct_16', 'oct_16', 'nov_16', 'nov_16']


In [23]:
print("IG for H_A in Out_NBC is:")
information_gain(H_A_nbc2, Label_nbc2, "Win", "Lose")
print("IG for Date in Out_NBC is:")
information_gain(Date_nbc2, Label_nbc2, "Win", "Lose")
print("IG for Opponent in Out_NBC is:")
information_gain(Opponent_nbc2, Label_nbc2, "Win", "Lose")

IG for H_A in Out_NBC is:
1.56093787441
IG for Date in Out_NBC is:
0.240355884586
IG for Opponent in Out_NBC is:
0.321663178886


This means we will split on Home/Away for Out_NBC. Thus we will create Out_NBC_Home and Out_NBC_Away.

In [24]:
print("IG for H_A in Out_ABC is:")
information_gain(H_A_abc2, Label_abc2, "Win", "Lose")
print("IG for Date in Out_ABC is:")
information_gain(Date_abc2, Label_abc2, "Win", "Lose")
print("IG for Opponent in Out_ABC is:")
information_gain(Opponent_abc2, Label_abc2, "Win", "Lose")

IG for H_A in Out_ABC is:
0.584183719779
IG for Date in Out_ABC is:
0.552034063122
IG for Opponent in Out_ABC is:
0.57368877302


We will split on Opponent for Out_ABC, since all the games in Out_ABC are Away so it makes no sense to split on Home/Away. This means we will create Out_ABC_Virginia, Out_ABC_Temple, Out_ABC_Pitt, Out_ABC_Texas, and Out_ABC_NCState. However, these each only contain one game, so we do not need to calculate the information gain or split any further.

In [25]:
print(Date_nbc2)

['sept_15', 'sept_15', 'oct_15', 'nov_15', 'nov_15', 'sept_16', 'sept_16', 'sept_16', 'oct_16', 'nov_16']


Now, let us return to the In branch that is being split on Media. We have In_NBC, In_ABC, and In_FOX, however In_FOX only has one game so has no need for any new branches. Within In_ABC there are only two games and both of them are losses, so we also have no need to split. 

In [26]:
print("IG for H_A in In_NBC is:")
information_gain(H_A_nbc, Label_nbc, "Win", "Lose")
print("IG for Date in In_NBC is:")
information_gain(Date_nbc, Label_nbc, "Win", "Lose")
print("IG for Opponent in In_NBC is:")
information_gain(Opponent_nbc, Label_nbc, "Win", "Lose")

IG for H_A in In_NBC is:
0.5
IG for Date in In_NBC is:
0.576617011622
IG for Opponent in In_NBC is:
0.62832077057


This means we will split on Opponent. Thus we will create In_NBC_GT, In_NBC_USC, In_NBC_Stanford, and In_NBC_VT. However, for each of these there is only one game, thus there is no reason to calculate the information gain after.

Now we return to the Out branch, and create the lists for Out_NBC_Home and Out_NBC_Away.

In [27]:
Label_home = []
Label_away = []
Date_home = []
Date_away = []
Opponent_home = []
Opponent_away = []
for index, item in enumerate(H_A_nbc2):
    if item == "Home":
        Date_home.append(Date_nbc2[index])
        Opponent_home.append(Opponent_nbc2[index])
        Label_home.append(Label_nbc2[index])
    else:
        Date_away.append(Date_nbc2[index])
        Opponent_away.append(Opponent_nbc2[index])
        Label_away.append(Label_nbc2[index])

In [28]:
print(Label_away)

['Win']


There is only one game in Out_NBC_Away, so we are done with this branch and do not need to calculate information gain. However, we do need to calculate information gain for Out_NBC_Home to decide what to split on next.

In [29]:
print("IG for Date in Out_NBC_Home is:")
information_gain(Date_home, Label_home, "Win", "Lose")
print("IG for Opponent in Out_NBC_Home is:")
information_gain(Opponent_home, Label_home, "Win", "Lose")

IG for Date in Out_NBC_Home is:
0.298659226585
IG for Opponent in Out_NBC_Home is:
0.349682813507


So now we will split on Opponent for Out_NBC_Home. However, this splits our tree into all individual games, so now there is no more information to be gained and our decision tree is done.

Now that we have split the tree until there is no information gain or all the features have been used, we can test our tree. For a visualization of the tree please see the PDF file. Below we will write the code to test our new data according to the decision tree branch rules. 

In [32]:
def predict_outcome(dataframe):
    predictions = []
    for index, row in dataframe.iterrows():
        if row["Is_Opponent_in_AP25_Preseason"] == "In":
            if row["Media"] == "Fox" or "ABC":
                predictions.append("Lose")
            elif row["Media"] == "NBC":
                if row["Opponent"] == "Virginia Tech" or "Stanford":
                    predictions.append("Lose")
                elif row["Opponent"] == "USC" or "Georgia Tech":
                    predictions.append("Win")
                else:
                    predictions.append("Win")
            else:
                predictions.append("Win")
        else:
            if row["Media"] == "CBS":
                predictions.append("Lose")
            elif row["Media"] == "ESPN":
                predictions.append("Win")
            elif row["Media"] == "ABC":
                if row["Opponent"] == "Virginia" or "Temple" or "PITT":
                    predictions.append("Win")
                elif row["Opponent"] == "Texas" or "North Carolina State":
                    predictions.append("Lose")
                else:
                    predictions.append("Win")
            elif row["Media"] == "NBC":
                if row["Opponent"] == "Texas" or "Army" or "UMass" or "Nevada" or "Wake Forest" or "Navy" or "Miami Florida":
                    predictions.append("Win")
                elif row["Opponent"] == "Michigan State" or "Duke":
                    predictions.append("Lose")
                else:
                    predictions.append("Win")
            else:
                predictions.append("Win")
    return predictions

In [34]:
predictions = predict_outcome(test)

In [51]:
actual = list(test["Label"])

In [59]:
precision_score(actual, predictions, pos_label="Win")

1.0

In [60]:
recall_score(actual, predictions, pos_label="Win")

0.88888888888888884

In [61]:
f1_score(actual, predictions, pos_label="Win")

0.94117647058823528